# Get data for new articles and list of all articles not citing data
A list of publications is obtainded from the app database. This list will contain a titles, IDs and DOIs which need to be explored to look for the corresponding pdf files. 
The steps of the process are: 
 1. get a Title, DOI, and URL for each publication
 2. convert the DOI to a pdf file name and try to open de file
 3. use pdfMiner and/or CDE to get the reference to data
 4. add a new dataset entry each time a new data object is found

In [23]:
# Libraries
# library containign functions that read and write to csv files
import lib.handle_csv as csvh
# library for connecting to the db
import lib.handle_db as dbh
# library for handling text matchings
import lib.text_comp as txtc
# library for getting data from crossref
import lib.crossref_api as cr_api
# library for handling url searchs
import lib.handle_urls as urlh
# managing files and file paths
from pathlib import Path
# add aprogress bar
from tqdm import tqdm_notebook 
from tqdm import tqdm
#library for handling json files
import json
# library for using regular expressions
import re
# library for handling http requests
import requests
# import custom functions (common to various notebooks)
import processing_functions as pr_fns

## Get pdf and HTML names into app DB

0. Add fields to articles table for holding pdf and html names
1. Open the previously verified DB and get the publications list
2. Open the current publication list from the appdb
3. Get pdf and html file names from previous and put it in current



In [25]:
# (0) Add fields
# ALTER TABLE articles 
# ADD COLUMN pdf_file text;
# ALTER TABLE articles 
# ADD COLUMN html_file text;

#(1) previously verified files:
prevapp_db = "db_files/app_db2.sqlite3"
while not Path(prevapp_db).is_file():
    print('Please enter the name of app db file:')
    prevapp_db = input()
    
# get publication data from the db
prev_pubs = pr_fns.get_pub_data(prevapp_db)
    
#2 currend app DB
ukchapp_db = "db_files/app_db20210702.sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()
    
# get publication data from the ukch app
app_pubs = pr_fns.get_pub_data(ukchapp_db)

# 3 get pdf and html name from previous and put it in current
for a_pub in tqdm_notebook(prev_pubs):
    pub_id = a_pub[0]
    pub_title = a_pub[1]
    pub_doi = a_pub[2]
    pub_url = a_pub[3]
    pub_pdf = a_pub[4]
    pub_html = a_pub[5]
    match_found = False
    for curr_pub in app_pubs:
        if curr_pub[2] == pub_doi and pub_doi != None:
            pr_fns.set_pdf_file_value(pub_pdf, curr_pub[0], ukchapp_db)
            match_found = True
            break
        elif curr_pub[1] == pub_title:
            pr_fns.set_pdf_file_value(pub_pdf, curr_pub[0], ukchapp_db)
            match_found = True
            break
    if not match_found:
        print("*******************\n",a_pub)   

C:\Users\scman1\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


## Check that pdf files exist 

Use the data on the articles table to verify if file are stored in the corresponding folder
We also check that the files in the folder are all accounted for (have a corersponding record)

In [32]:
# get publication data from the ukch app
app_pubs = pr_fns.get_pub_data(ukchapp_db)

i_indx = 1
for a_pub in tqdm_notebook(app_pubs):
    pub_id = a_pub[0]
    pub_title = a_pub[1]
    pub_doi = a_pub[2]
    pub_url = a_pub[3]
    pub_pdf = a_pub[4]
    pub_html = a_pub[5]
    if pub_pdf == None:
        print("*************************")
        print(i_indx, "Missing PDF for:", pub_doi, pub_id)
        i_indx +=1
    else:
        pdf_file = "pdf_files/" + pub_pdf
        if not Path(pdf_file).is_file():
            print("*************************")
            print(i_indx, "Missing file for:", pdf_file, "for", pub_doi, pub_id)
            i_indx +=1
            


C:\Users\scman1\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


*************************
1 Missing PDF for: 10.1002/9783527804085.ch10 64
*************************
2 Missing PDF for: 10.1016/b978-0-12-805324-9.09989-1 599
*************************
3 Missing PDF for: 10.1142/q0035 603
*************************
4 Missing PDF for: 10.1039/d0sc02253e 623
*************************
5 Missing PDF for: 10.1016/j.cell.2021.04.001 652
*************************
6 Missing PDF for: 10.1016/j.xcrp.2021.100349 653
*************************
7 Missing PDF for: 10.1039/D0CY01608J 654
*************************
8 Missing PDF for: 10.1021/acs.est.0c04279 655
*************************
9 Missing PDF for: 10.1021/jacsau.1c00145 656
*************************
10 Missing PDF for: 10.1039/D0CP01192D 657
*************************
11 Missing PDF for: 10.1021/acscatal.1c00997 658
*************************
12 Missing PDF for: 10.1007/s11244-018-0902-9 659
*************************
13 Missing PDF for: 10.1039/d1fd00027f 660
*************************
14 Missing PDF for: 10.1021/ac

In [40]:
for infile in tqdm_notebook(Path("pdf_files").glob('*.pdf')):
    file_found = False
    for a_pub in app_pubs:
        if infile.name == a_pub[4]:
            file_found = True
            break
    if not file_found:
        print("Not in DB:", infile.name)

C:\Users\scman1\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


Not in DB: 1-s2.0-S0926860X18301686-main.pdf
Not in DB: 287598212.pdf
Not in DB: Bahruji2015_Article_RutileTiO2PdPhotocatalystsForH.pdf.pdf
Not in DB: Bowker2015_Article_ThePhotocatalyticWindowPhoto-R.pdf.pdf
Not in DB: c6nr00053c.pdf
Not in DB: cs400683e.pdf.pdf
Not in DB: cs502038y.pdf.pdf
Not in DB: d0sc02253e1.pdf
Not in DB: s23.pdf
Not in DB: s5.pdf
Not in DB: telluriumdoped_lanthanum_manganite_as_catalysts_for_the_oxygen_reduction_reaction.pdf



In [ ]:
# functions for ChemDataExtractor
# not used for mining data references (suplementary/raw) or to get pdf metadata
from chemdataextractor import Document

# A function for getting a list of files from the directory
# This will be modified to get the list from a csv file
def get_files_list (source_dir):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob('*.pdf')):
        i_counter += 1
        files_list.append(filepath)
    return files_list

def cde_read_pdfs(a_file):
    pdf_f = open(a_file, 'rb')
    doc = Document.from_file(pdf_f)
    return doc

def find_doi(element_text):
    cr_re_01 = '10.\d{4,9}/[-._;()/:A-Z0-9]+'
    compare = re.search(cr_re_01, element_text, re.IGNORECASE)
    if compare != None:
        return compare.group()
    return ""

def get_db_id(doi_value, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    id_val = db_conn.get_value(table, "id", "doi", doi_value)
    db_conn.close()
    if id_val != None:
        return id_val[0]
    else:
        return 0

def get_db_title(doi_value, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    id_val = db_conn.get_value(table, "title", "doi", doi_value)
    db_conn.close()
    if id_val != None:
        return id_val[0]
    else:
        return 0

def get_close_dois(str_name, db_name = "prev_search.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    search_in = 'articles'
    fields_required = "id, doi, title, pdf_file"
    filter_str = "doi like '%"+str_name+"%';"

    db_titles = db_conn.get_values(search_in, fields_required, filter_str)
    db_conn.close()
    return db_titles

In [ ]:
import pdfminer
from pdfminer.high_level import extract_text

# functions for PDFminer

def get_pdf_text(pdf_file):
    return extract_text(pdf_file)

# get the paragraph fragments with references to data
def get_ref_sentences(pdf_text):
    sentences = pdf_text.split("\n")
    groups=[]
    for sentence in sentences:
        if pr_fns.is_data_stmt(sentence.lower()):
            idx = sentences.index(sentence)
            groups.append([idx-1,idx,idx+1])
    reduced_groups = []
    for group in groups:
        idx_group = groups.index(group)
        if groups.index(group) > 0:
            set_g = set(group)
            # make the array before current a set
            set_bg = set(groups[idx_group - 1])
            # make the array after current a set
            set_ag = set()
            if idx_group + 1 < len(groups):    
                set_ag = set(groups[idx_group + 1])
            if len(set_bg.intersection(set_g)) > 0:
                ordered_union = list(set_bg.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(set_ag.intersection(set_g)) > 0:
                ordered_union = list(set_ag.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(reduced_groups) > 0:
                is_in_rg = False
                for a_rg in reduced_groups:
                    if set_g.issubset(a_rg):
                        is_in_rg = True
                        break
                if not is_in_rg:
                    reduced_groups.append(list(set_g))
    return_group = []
    for sentence_group in reduced_groups:
        full_sentence = ""
        for single_sentence in sentence_group:
            full_sentence += sentences[single_sentence].strip()
        return_group.append(full_sentence)
    return return_group

# get the paragraph fragments with references to data
def get_all_data_sentences(pdf_text):
    sentences = pdf_text.split("\n")
    groups=[]
    for sentence in sentences:
        if 'data' in sentence.lower() or 'inform' in sentence.lower():
            idx = sentences.index(sentence)
            groups.append([idx-1, idx, idx+1])
    reduced_groups = []
    for group in groups:
        idx_group = groups.index(group)
        if groups.index(group) > 0:
            set_g = set(group)
            # make the array before current a set
            set_bg = set(groups[idx_group - 1])
            # make the array after current a set
            set_ag = set()
            if idx_group + 1 < len(groups):    
                set_ag = set(groups[idx_group + 1])
            if len(set_bg.intersection(set_g)) > 0:
                ordered_union = list(set_bg.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(set_ag.intersection(set_g)) > 0:
                ordered_union = list(set_ag.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(reduced_groups) > 0:
                is_in_rg = False
                for a_rg in reduced_groups:
                    if set_g.issubset(a_rg):
                        is_in_rg = True
                        break
                if not is_in_rg:
                    reduced_groups.append(list(set_g))
    return_group = []
    for sentence_group in reduced_groups:
        full_sentence = ""
        for single_sentence in sentence_group:
            full_sentence += sentences[single_sentence].strip()
        if not full_sentence in return_group:
            return_group.append(full_sentence)
    return return_group

# get the http strings from references to data
def get_http_ref(sentence):
    http_frag = ""
    if 'http' in sentence.lower():
        idx_http = sentence.lower().index('http')
        http_frag = sentence[idx_http:]
        space_in_ref = True
        while " " in http_frag:
            space_idx = http_frag.rfind(" ")
            http_frag = http_frag[:space_idx]
        if(http_frag[-1:]=="."):
            http_frag = http_frag[:-1]
    return http_frag

Get the name of the current app db file:

In [ ]:
# app db file with path: db_files/app_db.sqlite3
ukchapp_db = "db_files/app_db2.sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()
ukchapp_db



## Use pdfminer to get metadata from pdf file

In [ ]:
# get publication data from the ukch app
db_pubs = pr_fns.get_pub_app_data(ukchapp_db)

# get the list of dois already mined for data 
input_file = 'pub_data_add202012.csv'
id_field = 'num'
processed, headings = csvh.get_csv_data(input_file, id_field)
for id_num in processed:
    current_title = processed[id_num]['doi']
processed[1]['num']

processed_dois = []
for entry in processed:
    if not processed[entry]['doi'] in processed_dois:
        processed_dois.append( processed[entry]['doi'])

data_records = {}
data_mentions = {}
ref_count = mention_count = 0
for a_pub in tqdm_notebook(db_pubs):
    data_refs = []
    data_sents = []
    if a_pub[0] > 616:
        pub_id = a_pub[0]
        pub_title = a_pub[1]
        pub_doi = a_pub[2]
        pub_url = a_pub[3]
        pub_pdf = a_pub[4]
        pub_html = a_pub[5]
        if pub_pdf == 'None':
            print("*************************")
            print("Missing PDF for:", pub_doi)
            print("*************************")
        else:
            pdf_file = "pdf_files/" + pub_pdf
            if not Path(pdf_file).is_file():
                print("*************************")
                print("Missing file for:", pdf_file, "for", pub_doi)
                print("*************************")
            else: 
                print("PDF filename", pdf_file)
                pdf_text = get_pdf_text(pdf_file)
                ref_sentences = get_ref_sentences(pdf_text)
                data_sentences = get_all_data_sentences(pdf_text)
                for r_sentence in ref_sentences:
                    dt_link = get_http_ref(r_sentence)
                    if 'supplem' in r_sentence.lower():
                        data_refs.append({'type':'supplementary',"desc":r_sentence, 'data_url':dt_link})
                    else:
                        data_refs.append({'type':'supporting',"desc":r_sentence, 'data_url':dt_link})
                for d_sentence in data_sentences:
                    dt_link = get_http_ref(d_sentence)
                    if 'supplem' in d_sentence.lower():
                        data_sents.append({'type':'supplementary',"desc":d_sentence, 'data_url':dt_link})
                    else:
                        data_sents.append({'type':'supporting',"desc":d_sentence, 'data_url':dt_link})
        if data_refs != []:
            for data_ref in data_refs:
                data_record = {'id':pub_id, 'doi':pub_doi}    
                data_record.update(data_ref)
                data_records[ref_count] = data_record
                ref_count += 1
        if data_sents != []:
            for data_sent in data_sents:
                sentence_record = {'id':pub_id, 'doi':pub_doi}    
                sentence_record.update(data_sent)
                data_mentions[mention_count] = sentence_record
                mention_count += 1

In [ ]:
#if len(data_records) > 0:
#    csvh.write_csv_data(data_records, 'pdf_data.csv')
    
if len(data_mentions) > 0:
    csvh.write_csv_data(data_mentions, 'pdf_mentions202012.csv')

In [ ]:
# get names and links for references in data mentions
data_mentions, dm_fields = csvh.get_csv_data('pdf_mentions_filtered_02.csv', 'num')

for dm in data_mentions:
    print("https://doi.org/" + data_mentions[dm]['doi'])
    ref_name = data_mentions[dm]['ref_name']
    while ref_name == "":
        print('Please enter the name of data object:')
        ref_name = input()
    ref_link = data_mentions[dm]['ref_link']
    while ref_link == "":
        print('Please enter the data object link:')
        ref_link = input()
    data_mentions[dm]['ref_name'] = ref_name
    data_mentions[dm]['ref_link'] = ref_link
                      

In [ ]:
len(data_records)